In [1]:
import os.path
from sklearn.feature_extraction.text import CountVectorizer
from gensim.test.utils import common_texts, get_tmpfile
from sklearn.model_selection import GridSearchCV
from gensim.models import Word2Vec
from standard_function import *
from return_detail_icd_code import *
import pickle
import unicodedata
from fuzzywuzzy import process
from fuzzywuzzy import fuzz
import pandas as pd

In [2]:
load_data_check_default = pd.read_excel("dataset_train.xls", header = None, encoding = 'utf-8')
load_data_check = load_data_check_default.copy()
load_data_check[2] = load_data_check[2].apply(delete_special_char)

test_dataframe = pd.read_excel("again.xlsx", sheetname = "Dictionary_v2")
test_data_col = test_dataframe["Description"]

f = open('Eclaim_predict.pickle', 'rb')       # load model dự đoán mã icd 
classifier = pickle.load(f)
f.close()

with open ('exacly_2_word', 'rb') as fp_2:    # Load data về 2 cụm từ 
    words_2 = pickle.load(fp_2)
    
with open ('exacly_3_word', 'rb') as fp_3:    # Load data về 3 cụm từ
    words_3 = pickle.load(fp_3)
    
with open('link_words.pkl', 'rb') as f:
    listview = pickle.load(f)
    
model_2 = Word2Vec.load("2_gram.model")
model_3 = Word2Vec.load("3_gram.model")

def try_except_2_gram ( string):
    try :  
        model_2.wv.similar_by_word(string)
    except :
        kq = False
    else :
        kq = True
    return kq

def try_except_3_gram ( string):
    try :  
        model_3.wv.similar_by_word(string)
    except :
        kq = False
    else :
        kq = True
    return kq

dim_test_col = len(test_data_col)
var_run = 0
result_code = []

print("Running..........................!")

while var_run < dim_test_col:
    
    text = test_data_col[var_run]

    text = revert_text(out_special_char(text))    # hàm loại bỏ dấu ngoặc tròn, vuông , \ , /
    pr = spaces_num(char_num(text))    # loại bỏ khoản trắng vô nghĩa  
    pr1 = ' '.join(pr)                 # convert  list to string
    pr1 = delete_special_char(pr1)     # delete others special char --- > string return str

    text1 = pr1.strip("").split(" ")       # Convert str to list
    text1 = spaces_num(text1)                # loại bỏ khoản trắng thừa trong list
    text2 = pr1.strip("").split("*")       # Convert str to list Text 1 chứa list các từ đơn
    text2 = spaces_num(text2)                # Text2 chứa List cả cụm từ
    temp_02 = text1.copy()

    check_logic = True
    if (len(text1) >= 3):
        sta = incorrect_spelling(text1, words_2, words_3)
        text1 = sta.strip(' ').split(' ')
        text2 = sta.strip(' ').split('*')
    elif len(text1) == 2:
        kq = check_incorrect_2_word(text1,words_2)
        text1 = kq[0]
        check_logic = kq[1]
    if len(text1) == 1:
        pass

    new_str =''
    for i in range(0,len(text1)-1):
        new_str += text1[i] + ' ' + text1[i+1] + ","

    new_str = new_str.strip(',').split(",")
    addr = []
    chuoi = ''
    for index,i in enumerate (new_str[:len(new_str)]):
        if try_except_2_gram(i) == False :
            addr.append(index)
            chuoi += i + ","
    if len(text1)-len(addr) == 1:
        addr.append(max(addr) + 1)

    if len(addr) == 0:
        addr.append(-1)
    sum_index = addr[0]
    temp = []
    for i in range(0,len(addr)-1):
        if (addr[i+1] - addr[i]) == 1:
            sum_index = addr[i+1]
        else:
            temp.append(sum_index)
            sum_index = addr[i+1]
    temp.append(sum_index)
    if temp[0] == '':
        temp.append(len(text1))
    else:
        temp = list(set(temp))
        temp.sort(reverse = False)

    check = ''
    emty = ''
    sta = []
    sto = []
    sta1 = []
    for index, x in enumerate(text1):     # cắt chuỗi bệnh thành các loại bệnh bước 1
        if index in temp:
            check += x + ','
        else:
            check += x + ' '
    check = spaces_num(check.strip().split(','))

    if check_logic == False:             # Tách 2 bệnh chỉ có 2 chữ cái
        check = temp_02

    for i in range(0, len(check)):       # Gộp các từ đơn lẻ tránh lỗi Vocabulary
        if len(check[i]) == 1:
            check.remove(check[i])
            break

    for i in range(0, len(check)): 
        if len(check[i]) == 1:
            check[i-1] = check[i-1] + ' ' + check[i]
            check.remove(check[i])

    for i in check:                     # Tách các bệnh dính liền thành các loại bệnh bước 2
        ix = i.split(' ')
        for j in listview:
            if j in i:
                jx = j.split(' ')
                for index, val in enumerate(ix[0:len(ix)-2]):
                    if (ix[index] == jx[0]) and (ix[index+1]==jx[1]) and (try_except_3_gram(j+' '+ix[index+2]) == False):
                        ix.insert(index+1, ',') 
        ix = ' '.join(ix)
        i = ix
        emty += i +','
    check = emty.strip(',').split(',')

    with open ('sample_vector', 'rb') as fp:
        itemlist = pickle.load(fp)

    count = CountVectorizer()
    count.fit(itemlist)
    bag_of_words = count.transform(itemlist)

    x_new1 = count.transform(check)
    x_new1
    kq = classifier.predict(x_new1)

    dicti = []
    chieu_dai = len(kq)
    for g in range(0,chieu_dai):
        df_end = (load_data_check[load_data_check[1] == kq[g]])
        io = decission_value_return_01(df_end, check[g].split(','))

        Sam2Data = io[0].strip() + ' ' + str(io[1])
        dicti.append(Sam2Data)
    print(dicti,',')
    Sam2Data=''
    dicti = []
    var_run += 1
print("Completed !")

/home/administrator/.local/lib/python3.6/site-packages/pandas/util/_decorators.py:188: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  return func(*args, **kwargs)


Running..........................!
['0001 1.0'] ,
['0002 1.0'] ,
['0002 1.0'] ,
['0002 1.0'] ,
['0002 1.0'] ,
['B345 0.7', '0002 1.0'] ,
['0003 0.8'] ,
['Q752 0.45', 'C968 0.61', 'S020 0.75'] ,
['0019 0.52'] ,
['0006 0'] ,
['I99 0.68', '0019 0.42'] ,
['0008 1.0'] ,
['A085 0.76'] ,
['I50 0.58', '0010 0.78'] ,
['J983 0', '0011 1.0'] ,
['S924 0.57', 'D81 0.58', 'M895 0.83'] ,
['0009 0', '0013 0.58', '0002 0'] ,
['0013 0.73', '0013 0.71'] ,
['0013 0.68'] ,
['0013 0.57', '0013 0.76'] ,
['O821 0.39', '0013 0.71'] ,
['0013 0.93'] ,
['O300 0.44', 'R06 0.53'] ,
['H61 0.9', '0015 0.92', '0015 0.67'] ,
['0015 0.88'] ,
['Q624 0.69', '0011 0.66'] ,
['A03 1.0'] ,
['A04 0.97'] ,
['A05 0.86'] ,
['A059 0.97'] ,
['K511 0.72', 'K635 0.49'] ,
['A083 1.0'] ,
['A085 0.88'] ,
['A085 1.0'] ,
['K521 0.77'] ,
['A090 0.91'] ,
['A090 0.93'] ,
['K296 0.78', 'A090 0.87'] ,
['A090 0.84'] ,
['A163 1.0'] ,
['A18 1.0'] ,
['A41 1.0', 'K610 0.78', 'K622 0.7'] ,
['B46 0.52', 'Q667 0.69'] ,
['A418 1.0'] ,
['A418 1.0'] ,
['

['L304 0.45', 'N485 0.89'] ,
['K040 0.69', 'L120 0'] ,
['L02 1.0'] ,
['K61 0.65'] ,
['L022 1.0'] ,
['L02 0.46'] ,
['L732 0', 'N90 0.66'] ,
['L04 1.0'] ,
['G511 0.72', '0011 0.66'] ,
['L304 0.6', 'T784 0.85', 'B349 0.38'] ,
['L23 0.97'] ,
['L282 0.88'] ,
['B33 0.54'] ,
['L304 0.6'] ,
['L309 1.0'] ,
['L50 1.0', 'K050 0.69'] ,
['L50 1.0'] ,
['T07 0.39'] ,
['L84 1.0'] ,
['L958 0.81', 'T784 0.62'] ,
['K052 0.42'] ,
['M772 0.77'] ,
['M772 0.77'] ,
['M13 1.0'] ,
['M13 0.79'] ,
['M13 0.59'] ,
['M130 0.89'] ,
['M130 1.0'] ,
['M131 0.79', 'K050 0.69'] ,
['M17 1.0'] ,
['M17 1.0'] ,
['S630 0.57'] ,
['M254 0.51'] ,
['M254 1.0'] ,
['M47 0.86'] ,
['M47 1.0'] ,
['M478 1.0'] ,
['M47 0.71'] ,
['M483 1.0', 'M483 0.86', '0007 0'] ,
['K591 0', 'M50 0.94'] ,
['M501 0.96', 'M51 0.81'] ,
['M502 1.0'] ,
['Q330 0', 'M503 1.0'] ,
['M51 0.93'] ,
['M51 1.0'] ,
['M511 0.77'] ,
['M511 0.9'] ,
['M51 0.81', 'G51 0.56'] ,
['M531 0.88'] ,
['M530 0.9', 'G830 0.46'] ,
['E241 0.86', 'M542 0.47'] ,
['M542 0.66', 'E11 0'] ,


['S060 1.0'] ,
['S20 1.0'] ,
['S400 1.0'] ,
['S525 0.94'] ,
['S61 0.95'] ,
['S90 0.95'] ,
['S91 0.95'] ,
['T11 0.99'] ,
['T71 1.0'] ,
['T784 1.0'] ,
Completed !
